# **Introduction of Project ETL with Python**

Extract, Transform and Load (ETL) is a collection of processes to "move" data from one place to another.
The place in question is from data sources (can be application databases, files, logs, databases from 3rd party, and others) to the data warehouse.

What is a data warehouse?

In short, a data warehouse is a database that contains data (tables) that are ready for analysis by Data Engineer and Data Scientists.

More details can be seen at:
https://en.wikipedia.org/wiki/Data_warehouse.

In this Project I will Practice each of these processes.

# Projects to be Done
In this project, I will process the registrant data for a hackathon organized by DQThon.

This dataset consists of 5000 rows of data (5000 registrants) in CSV (Comma-separated values) format and has several columns including:

- participant_id: ID of the participant/participant of the hackathon. This column is unique so each participant must have a different ID
- first_name: participant's first name
- last_name: participant's last name
- birth_date: participant's date of birth
- address: participant's residence address
- phone_number: participant's cellphone/phone number
- country: participant's country of origin
- institute: the current participating institution, can be the name of the company or the name of the university
- occupation: participant's current job
- register_time: the time participants register for the hackathon in seconds

However, in this project I will be asked to generate several columns by utilizing the existing columns, so that the end of this project is the result of data transformation with several new columns apart from the 10 columns above.

As a warm-up to this project, I will open the contents of the dataset and see the values.



---



---



# **Extract**
Extract is the process of extracting data from sources, this data source can be relational data (SQL) or tables, non-relational (NoSQL) or others.

My task is to first read this dataset as a CSV so that it can be processed later. 

The file can be accessed via the URL: https://storage.googleapis.com/dqlab-dataset/dqthon-participants.csv.

In [1]:
import pandas as pd
df_participant = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/dqthon-participants.csv')



---



---





# **Transform**
Transform is the process of transforming data, or changes to data. Generally like:

- Change the value of a column to a new value,
- Creates a new column by making use of another column,
- Transpose rows to columns (or vice versa),
- Change the data format to a more standard form (for example, date and datetime columns which usually have non-standard values or HP numbers which usually have values that do not match the standard format), and others.

## Transform Part I - Postal Code

There was a request from the logistics team that they needed a postal code from the participant so that the delivery of the trophy was easier and faster. Therefore, create a new column called postal_code which contains information about the postal code taken from the participant's address (address column).

Note that the postal code is at the very end of the address.

In [2]:
df_participant['postal_code'] = df_participant['address'].str.extract(r'(\d+)$')

## Transform Part II - City
In addition to the postal code, they also need the city of the participant.

To provide this information, create a new column named city which is obtained from the address column. It is assumed that the city is a set of characters after the street number followed by \n (newline character) or in other languages, namely enter.

In [3]:
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)') 

## Transform Part III - Github
One of the parameters to find out what projects the participants have worked on is from their git repository.

In this case we use the github profile as the parameter. Your task is to create a new column named github_profile which is the github profile link of the participant.

Note that their github profile is a concatenation of lowercase first_name and last_name.

In [4]:
df_participant['github_profile'] = 'https://github.com/' + df_participant['first_name'].str.lower() + df_participant['last_name'].str.lower()

# Transform Part IV - Mobile Number
If we look back, it turns out that the cellphone numbers in our csv data have different formats. Therefore, we need to clean the mobile number data so that it has the same format. You as a Data Engineer are given the privilege to determine the correct mobile number format. In this case let's equate the format with the rule:

- If the prefix of the mobile number is the number 62 or +62 which is the Indonesian telephone code, it will be translated to 0.
- No punctuation such as opening parenthesis, closing parenthesis, strip⟶ ()-
- There is no space in the column name cell phone number to store the cleaning results on the cellphone number, namely cleaned_phone_number

In [6]:
df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


# Transform Part V - Team Name
The dataset does not currently contain team names, and apparently the Data Analyst team requires information regarding the team names of each participant.

Note that the team name is a combination of values from the first_name, last_name, country and institute columns.

My task is to create a new column with the name team_name which contains the team name information from the participants.

In [7]:
def func(col):
    abbrev_name = "%s%s"%(col['first_name'][0],col['last_name'][0]) #Abbreviation of First Name and Last Name by taking the first letter
    country = col['country']
    abbrev_institute = '%s'%(''.join(list(map(lambda word: word[0], col['institute'].split())))) #Abbreviation of value in the institute column
    return "%s-%s-%s"%(abbrev_name,country,abbrev_institute)

df_participant['team_name'] = df_participant.apply(func, axis=1)

# Transform Part VI - Email
After reviewing the participant data, it turns out that there is one important information that is not yet available, namely email.

I am asked to provide email information from participants with the rules that the email format is as follows:

---

Email Formats:

xxyy@aa.bb.[ac/com].[cc]

Information:
- xx -> first name (first_name) in lowercase
- yy -> last name (last_name) in lowercase
- aa -> institution name

For the value of bb, and cc follow the value of aa. The rules:
- If the institution is a university, then
  bb -> a combination of the first letters of each word of the University name in lowercase
  Then, followed by .ac which indicates the academy/institution of learning and followed by pattern cc
- If the institution is not a University, then
  bb -> a combination of the first letters of each word of the University name in lowercase
  Then, followed by .com. Please note that the cc pattern does not apply in this condition

cc -> is the participant's country of origin, as for the rules:
- If the number of words in the country is more than 1 then take the abbreviation of that country in lowercase
- However, if the number of words is only 1 then take the first 3 letters of the country in lowercase

Example:
  First name: Citra;
  Last name: Nurdiyanti;
  Institution: UD Prakasa Mandasari;
  Country: Georgia.
  So, his email: citranurdiyanti@upm.geo
  --------------------------------------------------
  First name: Aris;
  Last name: Setiawan;
  Institution: Diponegoro University;
  Country: North Korea.
  
  So, Email: arissetiawan@ud.ac.ku

In [8]:
def func(col):
    first_name_lower = col['first_name'].lower()
    last_name_lower = col['last_name'].lower()
    institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split()))) #Abbreviation of company name in lowercase

    if 'Universitas' in col['institute']:
        if len(col['country'].split()) > 1: #Condition to check if the word count of country is more than 1
            country = ''.join(list(map(lambda word: word[0], col['country'].lower().split())))
        else:
            country = col['country'][:3].lower()
        return "%s%s@%s.ac.%s"%(first_name_lower,last_name_lower,institute,country)

    return "%s%s@%s.com"%(first_name_lower,last_name_lower,institute)

df_participant['email'] = df_participant.apply(func, axis=1)

# Transform Part VII - Date of Birth
MySQL is a database that is very popular and is used to store data in the form of tables, including the data from the processing that we have done, which can later be entered into MySQL.

Even so, there is a rule from MySQL regarding the date format they can accept, namely YYYY-MM-DD with the caption:

- YYYY: 4 digits indicating year
- MM: 2 digits indicating month
- DD: 2 digits indicating the date

Examples are: 2021-04-07

If we look again at the date of birth column, we can see that the value does not match the DATE format from MySQL.

Therefore, format the birth_date column to YYYY-MM-DD and save it in the same column.

In [9]:
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format='%d %b %Y')

# Transform Part VII - Competition Registration Date
In addition to having rules regarding the DATE format, MySQL also provides rules for data of type DATETIME, namely YYYY-MM-DD HH:mm:ss with the description:

- YYYY: 4 digits indicating year
- MM: 2 digits indicating month
- DD: 2 digits indicating the date
- HH: 2 digits indicating the hour
- mm: 2 digits indicating the minute
- ss: 2 digits indicating seconds

Examples are: 2021-04-07 15:10:55

Because our data regarding participant registration time (register_time) is not in the proper format.

Therefore, My task is to change register_time to DATETIME format according to MySQL's rules.

Save the results to the register_at column.

In [10]:
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')



---



---




# **Conclusion**
With that, I come to the end of the Transform section.

If we look back, my current dataset is different from the previous extract process. There are some additional columns that take advantage of other column values.

My dataset currently contains the following columns:

1. participant_id: ID of the participant/participant of the hackathon. This column is unique so each participant must have a different ID
2. first_name: participant's first name
3. last_name: participant's last name
4. birth_date: participant's date of birth (already formatted to YYYY-MM-DD)
5. address: participant's residence address
6. phone_number: participant's cellphone/phone number
7. country: participant's country of origin
8. institute: the current participating institution, can be the name of the company or the name of the university
9. occupation: participant's current job
10. register_time: the time participants register for the hackathon in seconds
11. team_name: name of the participating team (combination of first name, last name, country and institution)
12. postal_code: participant's address postal code (taken from the address field)
13. city: participant city (taken from the address column)
14. github_profile: link of participant's github profile (combination of first name and last name)
15. email: participant's email address (combination of first name, last name, institution and country)
16. cleaned_phone_number: participant's cellphone/telephone number (already more in line with the phone number format)
17. register_at: date and time participants registered (already in DATETIME format)



---



---



# Load
In this load section, the data that has been transformed in such a way that it fits the needs of the analyst team is entered back into the database, namely the Data Warehouse (DWH). Usually, the database schema is defined first, such as:

Column name
Column type
Is it primary key, unique key, index or not
Column length
Because generally Data Warehouses are structured databases so they need a schema before the data is entered.

Pandas already provides a function to enter data into the database, namely to_sql().

Details of the function can be found in the Pandas documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html